In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import *

# Read Dataset

In [2]:
df = pd.read_csv("energy_consumption.csv")
df.head()

,Datetime,DOM_MW
0,2005-12-31 01:00:00,9389.0
1,2005-12-31 02:00:00,9070.0
2,2005-12-31 03:00:00,9001.0
3,2005-12-31 04:00:00,9042.0
4,2005-12-31 05:00:00,9132.0


In [3]:
df = pd.read_csv("energy_consumption.csv", index_col = 'Datetime' , parse_dates = ['Datetime'])
df.head()

,DOM_MW
Datetime,
2005-12-31 01:00:00,9389.0
2005-12-31 02:00:00,9070.0
2005-12-31 03:00:00,9001.0
2005-12-31 04:00:00,9042.0
2005-12-31 05:00:00,9132.0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 116189 entries, 2005-12-31 01:00:00 to 2018-01-02 00:00:00
Data columns (total 1 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   DOM_MW  116189 non-null  float64
dtypes: float64(1)
memory usage: 1.8 MB


In [5]:
df.shape

(116189, 1)

# Data Preprocessing

In [6]:
def scaling(df):
    scaler = MinMaxScaler()
    df['DOM_MW'] = scaler.fit_transform(df['DOM_MW'].values.reshape(-1,1))
    return df

In [7]:
scaled_df = scaling(df)

In [8]:
scaled_df.shape

(116189, 1)

In [9]:
def data_preprocessing(data, time_step):
    x_train = []
    y_train = []
    
    for i in range(time_step, len(data)):
        x_train.append(data.iloc[i - time_step:i , 0])
        y_train.append(data.iloc[i,0])
        
    x_test = x_train[110000:]
    y_test = y_train[110000:]
    
    x_train = x_train[:110000]
    y_train = y_train[:110000]
    
    x_test = np.array(x_test)
    y_test = np.array(y_test)
    x_train = np.array(x_train)
    y_train = np.array(y_train)
    
    x_train = np.reshape(x_train,(110000, time_step, 1)) 
    x_test = np.reshape(x_test, (x_test.shape[0], time_step, 1))
    
    return [x_train, y_train, x_test, y_test]

In [10]:
time_step = 50
x_train, y_train, x_test, y_test = data_preprocessing(scaled_df, time_step)

In [11]:
print(x_train.shape)
print(y_train.shape)

(110000, 50, 1)
(110000,)


In [12]:
print(x_test.shape)
print(y_test.shape)

(6139, 50, 1)
(6139,)


# RNN Model

In [13]:
rnn_model = Sequential()

rnn_model.add(SimpleRNN(40, activation = 'tanh', return_sequences = True,
                       input_shape = (x_train.shape[1], 1)))

rnn_model.add(Dropout(0.15))

rnn_model.add(SimpleRNN(40, activation = 'tanh', return_sequences = True))

rnn_model.add(Dropout(0.15))

rnn_model.add(SimpleRNN(40, activation = 'tanh', return_sequences = True))

rnn_model.add(Dropout(0.15))

rnn_model.add(Dense(1))
    

# Model Compile & Fit

In [14]:
rnn_model.compile(optimizer = 'adam', loss = 'MSE')

In [15]:
rnn_model.fit(x_train, y_train, epochs = 10)

Epoch 1/10
3438/3438 [==============================] - 71s 20ms/step - loss: 0.0222
Epoch 2/10
3438/3438 [==============================] - 73s 21ms/step - loss: 0.0142
Epoch 3/10
3438/3438 [==============================] - 81s 24ms/step - loss: 0.0140
Epoch 4/10
3438/3438 [==============================] - 84s 25ms/step - loss: 0.0139
Epoch 5/10
3438/3438 [==============================] - 105s 31ms/step - loss: 0.0139
Epoch 6/10
3438/3438 [==============================] - 96s 28ms/step - loss: 0.0139
Epoch 7/10
3438/3438 [==============================] - 105s 31ms/step - loss: 0.0139
Epoch 8/10
3438/3438 [==============================] - 101s 29ms/step - loss: 0.0139
Epoch 9/10
3438/3438 [==============================] - 123s 36ms/step - loss: 0.0139
Epoch 10/10
3438/3438 [==============================] - 132s 38ms/step - loss: 0.0139


# Model Prediction

In [16]:
rnn_pred = rnn_model.predict(x_test)
rnn_pred

192/192 [==============================] - 3s 10ms/step


array([[[0.4717713 ],
        [0.4724102 ],
        [0.47319564],
        ...,
        [0.47363126],
        [0.47323966],
        [0.47311673]],

       [[0.47194678],
        [0.4727041 ],
        [0.47366497],
        ...,
        [0.4731385 ],
        [0.47318447],
        [0.47317848]],

       [[0.47229278],
        [0.473203  ],
        [0.4743135 ],
        ...,
        [0.4730939 ],
        [0.4732404 ],
        [0.47332692]],

       ...,

       [[0.47644386],
        [0.47533917],
        [0.474659  ],
        ...,
        [0.47600237],
        [0.4757037 ],
        [0.47571594]],

       [[0.47628483],
        [0.47510332],
        [0.47444534],
        ...,
        [0.4758906 ],
        [0.47563118],
        [0.47554323]],

       [[0.47599983],
        [0.47487587],
        [0.47439995],
        ...,
        [0.4758082 ],
        [0.47546318],
        [0.475442  ]]], dtype=float32)

# LSTM Model

In [18]:
lstm_model = Sequential()

lstm_model.add(LSTM(40, activation = 'tanh', return_sequences = True,
                   input_shape= (x_train.shape[1],1)))
lstm_model.add(Dropout(0.15))

lstm_model.add(LSTM(40, activation = 'tanh', return_sequences = True))
lstm_model.add(Dropout(0.15))

lstm_model.add(LSTM(40, activation = 'tanh', return_sequences = True))
lstm_model.add(Dropout(0.15))

lstm_model.add(Dense(1))

# Model Compile & Fit

In [19]:
lstm_model.compile(optimizer = 'adam', loss ='MSE')
lstm_model.fit(x_train, y_train, epochs = 10)

Epoch 1/10
3438/3438 [==============================] - 211s 59ms/step - loss: 0.0154
Epoch 2/10
3438/3438 [==============================] - 238s 69ms/step - loss: 0.0140
Epoch 3/10
3438/3438 [==============================] - 229s 67ms/step - loss: 0.0139
Epoch 4/10
3438/3438 [==============================] - 223s 65ms/step - loss: 0.0139
Epoch 5/10
3438/3438 [==============================] - 233s 68ms/step - loss: 0.0139
Epoch 6/10
3438/3438 [==============================] - 210s 61ms/step - loss: 0.0139
Epoch 7/10
3438/3438 [==============================] - 197s 57ms/step - loss: 0.0139
Epoch 8/10
3438/3438 [==============================] - 184s 54ms/step - loss: 0.0139
Epoch 9/10
3438/3438 [==============================] - 209s 61ms/step - loss: 0.0139
Epoch 10/10
3438/3438 [==============================] - 208s 61ms/step - loss: 0.0139


In [20]:
lstm_pred = lstm_model.predict(x_test)
lstm_pred

192/192 [==============================] - 3s 13ms/step


array([[[0.47030148],
        [0.4703003 ],
        [0.4702678 ],
        ...,
        [0.469917  ],
        [0.46991682],
        [0.46991336]],

       [[0.47030565],
        [0.47031164],
        [0.47028965],
        ...,
        [0.46991682],
        [0.46991336],
        [0.46991187]],

       [[0.47031382],
        [0.47033226],
        [0.4703254 ],
        ...,
        [0.4699134 ],
        [0.46991187],
        [0.46991253]],

       ...,

       [[0.47041336],
        [0.47050017],
        [0.4705211 ],
        ...,
        [0.47175187],
        [0.47175857],
        [0.47176152]],

       [[0.4704093 ],
        [0.47048923],
        [0.4705029 ],
        ...,
        [0.47175854],
        [0.47176152],
        [0.47175917]],

       [[0.47040215],
        [0.4704747 ],
        [0.47048488],
        ...,
        [0.4717615 ],
        [0.47175914],
        [0.47175175]]], dtype=float32)